## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('../')
from src import sat_download
from utils import post_utils
from utils import calib_utils
from utils import config_utils
from utils import data_utils
from utils import model_utils
from utils import plot_utils

%load_ext autoreload
%autoreload 2

/anaconda/envs/gigav3/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Load Config Files

In [2]:
iso_code = "TJK"
sum_threshold=-1
buffer_size=25
calibrator=None

data_config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AS.yaml")
data_config = config_utils.load_config(data_config_file)

## Load Model Predictions

In [3]:
preds = post_utils.load_preds(
    iso_code, data_config, sum_threshold, buffer_size, calibrator
)
preds.head(2)

Reading TJK_Zafarobod District_convnext_large_gradcam.geojson...: 100%|████████████████████| 14/14 [00:04<00:00,  3.01it/s]        
INFO:root:Expecting value: line 1 column 1 (char 0)
INFO:root:https://www.geoboundaries.org/api/current/gbHumanitarian/TJK/ADM3/
INFO:root:Data dimensions: (6228, 8)


,prob,UID,sum,geometry,group,ADM1,ADM2,PUID
0,0.978781,99911,17112.441033,POINT (7838136.418 4907515.151),4108,Sughd Region,Konibodom District,Konibodom District_99911
1,0.977980,102661,9690.999426,POINT (7839487.618 4909615.151),4132,Sughd Region,Konibodom District,Konibodom District_102661


In [ ]:
post_utils.save_results(iso_code, preds, source="preds")

## Load Government Data

In [259]:
master = post_utils.load_master(iso_code, data_config)
master.head(2)

INFO:root:clean
0    9958
1    1771
2     722
Name: count, dtype: int64
INFO:root:Data dimensions: (12451, 13)


,MUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,UNICEF-MOZ-SCHOOL-00000000,UNICEF,MOZ,Mozambique,Africa,Sub-Saharan Africa,None,0,0,POINT (35.10153 -17.41236),Tete,Mutarara,Nhamayabue
1,UNICEF-MOZ-SCHOOL-00000001,UNICEF,MOZ,Mozambique,Africa,Sub-Saharan Africa,None,1,0,POINT (36.91100 -14.50489),Niassa,Metarica,Nacumua


In [ ]:
post_utils.save_results(iso_code, master, source="master")

## Load OSM and Overture Data

In [260]:
osm_overture = post_utils.load_osm_overture(iso_code, data_config)
osm_overture.head(2)

INFO:root:clean
0    487
Name: count, dtype: int64
INFO:root:Data dimensions: (487, 13)


,SUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,OSM-MOZ-SCHOOL-00000000,OSM,MOZ,Mozambique,Africa,Sub-Saharan Africa,primary school,0,0,POINT (34.75546 -19.64221),Sofala,Dondo,Dondo
1,OSM-MOZ-SCHOOL-00000001,OSM,MOZ,Mozambique,Africa,Sub-Saharan Africa,Maputo International School,1,0,POINT (32.58823 -25.97934),Maputo City,Cidade De Maputo,Cidade De Maputo


In [ ]:
post_utils.save_results(iso_code, osm_overture, source="osm_overture")

## Standardize Data

In [270]:
preds = post_utils.standardize_data(iso_code, source="preds", uid="UID")
master = post_utils.standardize_data(iso_code, source="master", uid="MUID")
osm = post_utils.standardize_data(iso_code, source="osm_overture", uid="SUID")
preds.head(3)

,giga_id_school,school_id,name,lat,lon,admin1,admin1_id_giga,admin2,admin2_id_giga,education_level,source,uninhabited,is_duplicated,predicted_proba,rurban
0,ML-MOZ-SCHOOL-76580000,ML-MOZ-SCHOOL-76580000,Unknown,-19.124442,33.470237,Manica,MOZ006,Cidade de Chimoio,MOZ006001,None,ML,None,None,0.980880,urban
1,ML-MOZ-SCHOOL-24801000,ML-MOZ-SCHOOL-24801000,Unknown,-19.772812,34.876675,Sofala,MOZ007,Cidade da Beira,MOZ007001,None,ML,None,None,0.980790,urban
2,ML-MOZ-SCHOOL-21286000,ML-MOZ-SCHOOL-21286000,Unknown,-19.748836,34.851200,Sofala,MOZ007,Cidade da Beira,MOZ007001,None,ML,None,None,0.979622,urban


In [ ]:
data = pd.concat([preds, master, osm])
data = data.reset_index(drop=True)
data.to_parquet(f'{iso_code}_validator_meta.parquet')